In [ ]:
'''
10.3 한국어 임베딩
'''

In [ ]:
# 10-52 라이브러리 불러오기기
import torch
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

c:\Users\a0107\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\a0107\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [4]:
# 10-53 문장의 토크나이징
text = "나는 파이토치를 이용한 딥러닝을 학습중이다."
marked_text = "[CLS] " + text + " [SEP]" # 문장의 시작은 [CLS], 문장의 끝은 [SEP]로 형태를 맞춤.
tokenized_text = tokenizer.tokenize(marked_text) # 사전 훈련된 버트 토크나이저를 이용해서 문장을 단어로 쪼갬.
print(tokenized_text)

['[CLS]', '나는', '파', '##이', '##토', '##치를', '이', '##용한', '딥', '##러', '##닝', '##을', '학', '##습', '##중', '##이다', '.', '[SEP]']


In [ ]:
'''
쪼개진 단어들이 부정확함.
--> 버트 토크나이저가 단어의 가장 작은 조각을 기준으로 쪼개도록 설계되었기 때문.

영어의 경우 단어 기준으로 쪼개지지만 영어가 아닌 경우 토크나이징이 부정확할 수 있기 때문에
KoBert같은 국내에서 개발한 모델들을 이용하는 것도 좋음.
'''

In [5]:
# 10-54 모델을 훈련시킬 텍스트 정의
text = "과수원에 사과가 많았다." \
        "친구가 나에게 사과했다." \
        "백설공주는 독이 든 사과를 먹었다."
marked_text = "[CLS] " + text + " [SEP]"
tokenized_text = tokenizer.tokenize(marked_text) # 문장을 토큰으로 분리
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰 문자열에 인덱스를 매핑
for tup in zip(tokenized_text, indexed_tokens): # 단어와 인덱스를 출력
    print('{:<12} {:>6}'.format(tup[0], tup[1]))

[CLS]           101
과              8898
##수           15891
##원에         108280
사              9405
##과           11882
##가           11287
많              9249
##았다          27303
.               119
친              9781
##구           17196
##가           11287
나              8982
##에게          26212
사              9405
##과           11882
##했다          12490
.               119
백              9331
##설           31928
##공           28000
##주는         100633
독              9088
##이           10739
든              9115
사              9405
##과           11882
##를           11513
먹              9266
##었다          17706
.               119
[SEP]           102


In [ ]:
'''
버트는 문장을 구별하기 위해 1과 0을 사용.
즉, 문장이 바뀔 때마다 0 -> 1로 바뀌고, 또 다시 문장이 바뀌면 1 -> 0으로 바뀜.
ex) [0, 0, 1, 1, 1, 0, 0, 0]: 세 개의 문장으로 구성됨.
'''

In [6]:
# 10-55 문장 인식 단위 지정
segments_ids = [1] * len(tokenized_text)
print(segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [7]:
# 10-56 데이터를 텐서로 변환
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [8]:
# 10-57 모델 생성
model = BertModel.from_pretrained('bert-base-multilingual-cased',
                                  output_hidden_states=True) # 12개의 계층으로 구성된 DNN
model.eval() # 모델을 평가 모드로 설정

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [9]:
# 10-58 모델 훈련
with torch.no_grad(): # 모델을 평가할 때는 기울기를 계산하지 않음음.
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2] # 네트워크의 은닉 상태를 가져옴.

In [10]:
# 10-59 모델의 은닉 상태 정보 확인
print("계층 개수:", len(hidden_states), "   (initial embeddings + 12 BERT layers)")
layer_i = 0
print("배치 개수:", len(hidden_states[layer_i]))
batch_i = 0
print("토큰 개수:", len(hidden_states[layer_i][batch_i]))
token_i = 0
print("은닉층의 유닛 개수:", len(hidden_states[layer_i][batch_i][token_i]))

계층 개수: 13    (initial embeddings + 12 BERT layers)
배치 개수: 1
토큰 개수: 33
은닉층의 유닛 개수: 768


In [ ]:
# 네 개의 차원(계층, 배치, 토큰, 은닉층 유닛) -> 세 개의 차원(토큰, 계층, 은닉층 유닛)

In [11]:
# 10-60 모델의 은닉 상태 정보 확인
print('은닉 상태의 유형: ', type(hidden_states))
print('각 계층에서의 텐서 형태: ', hidden_states[0].size())

은닉 상태의 유형:  <class 'tuple'>
각 계층에서의 텐서 형태:  torch.Size([1, 33, 768])


In [12]:
# 10-61 텐서의 형태 변경
token_embeddings = torch.stack(hidden_states, dim=0) # 각 계층의 텐서 결합은 stack 사용
token_embeddings.size() # 최종 텐서의 형태 출력력

torch.Size([13, 1, 33, 768])

In [13]:
# 10-62 텐서의 형태 변경
token_embeddings = torch.squeeze(token_embeddings, dim=1) # 최종 텐서에서 불필요한 '배치' 차원(1) 제거
token_embeddings.size() # 배치 차원 제거 후 최종 텐서의 형태를 출력력

torch.Size([13, 33, 768])

In [14]:
# 10-63 텐서 차원 변경
token_embeddings = token_embeddings.permute(1, 0, 2) # '계층 개수', '토큰 개수'에 대한 차원 위치 바꿈
token_embeddings.size()

torch.Size([33, 13, 768])

In [16]:
# 10-64 각 단어에 대한 벡터 형태 확인
token_vecs_cat = [] # 형태가 [33*(33*768)]인 벡터를 [33*25.344]로 변경하여 저장
for token in token_embeddings: # token_embeddings는 [33*12*768] 형태의 텐서를 가짐.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    token_vecs_cat.append(cat_vec)
print('형태는: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

# 개별적 토큰은 [12 * 768]의 형태를 가지며, 네 개의 계층을 이어 붙이면 각 계층의 벡터는 768개의 값을 갖게 되므로 'cat_vec'의 길이는 3072가 됨.
# 'cat_vec'는 토큰을 의미미

형태는: 33 x 3072


In [17]:
# 10-65 계층을 결합하여 최종 단어 벡터 생성
token_vecs_sum = [] # [33*768] 형태의 토큰을 벡터로 저장
for token in token_embeddings: # 'token_embeddings'는 [33*12*768] 형태의 토큰을 가짐.
    sum_vec = torch.sum(token[-4:], dim=0) # 마지막 네 개 계층의 벡터를 합산
    token_vecs_sum.append(sum_vec) # 'sum_vec'을 사용하여 토큰을 표현
print('형태는: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

형태는: 33 x 768


In [18]:
# 10-66 문장 벡터
token_vecs = hidden_states[-2][0]
sentence_embedding = torch.mean(token_vecs, dim=0)
print("최종 임베딩 벡터의 형태:", sentence_embedding.size())

최종 임베딩 벡터의 형태: torch.Size([768])


In [19]:
# 10-67 토큰과 인덱스 출력
for i, token_str in enumerate(tokenized_text):
    print(i, token_str)

0 [CLS]
1 과
2 ##수
3 ##원에
4 사
5 ##과
6 ##가
7 많
8 ##았다
9 .
10 친
11 ##구
12 ##가
13 나
14 ##에게
15 사
16 ##과
17 ##했다
18 .
19 백
20 ##설
21 ##공
22 ##주는
23 독
24 ##이
25 든
26 사
27 ##과
28 ##를
29 먹
30 ##었다
31 .
32 [SEP]


In [20]:
# 10-68 단어 벡터 확인
print("사과가 많았다", str(token_vecs_sum[6][:5]))
print("나에게 사과했다", str(token_vecs_sum[10][:5]))
print("사과를 먹었다", str(token_vecs_sum[19][:5]))

# '사과'가 붙어있지 않고 쪼개진 상태일 때 벡터

사과가 많았다 tensor([-0.5844, -4.0836,  0.4906,  0.8915, -1.8054])
나에게 사과했다 tensor([-0.8631, -3.4047, -0.7351,  0.9805, -2.6700])
사과를 먹었다 tensor([ 0.6756, -0.3618,  0.0586,  2.2050, -2.4193])


In [23]:
# 10-69 코사인 유사도 계산
from scipy.spatial.distance import cosine
diff_apple = 1 - cosine(token_vecs_sum[5], token_vecs_sum[27]) # '사과가 많았다'와 '나에게 사과했다'에서 단어 '사과' 사이의 코사인 유사도를 계산
same_apple = 1 - cosine(token_vecs_sum[5], token_vecs_sum[16]) # '사과가 많았다', '사과를 먹었다'에 있는 '사과' 사이의 코사인 유사도를 계산
print('*유사한* 의미에 대한 벡터 유사성: %.2f' % same_apple)
print('*다른* 의미에 대한 벡터 유사성: %.2f' % diff_apple)

*유사한* 의미에 대한 벡터 유사성: 0.86
*다른* 의미에 대한 벡터 유사성: 0.91
